In [ ]:
from IPython.core.display import HTML
with open('style.css') as file:
    css = file.read()
HTML(css)

In [ ]:
# Autload python modules by default
%load_ext autoreload
%autoreload 2

# Convert notebooks to python, so they can be loaded effiently
from utils.jupyter_loader import JupyterLoader

loader = JupyterLoader()
loader.load_all()

# Play Game

We already developed a small method to let two engines play against each other in the previous chapter. The goal is to write a more sophisticated method, which is able to:
* Optionally display the board after each move 
* Optionally write the game in a file

To display the board after each move a helper function `display_chess_board` is defined. As a parameter it takes the current chess board `board` and a boolean flag `display`. If `display` is set to `False` the method will do nothing, otherwise it clears the current output, displays the board and waits half a second, so the animation is fluent when the method is called multiple times.

In [ ]:
import chess
import time
import IPython.display


def display_chess_board(board: chess.Board, display: bool = True):
    if display:
        IPython.display.clear_output(wait=True)
        IPython.display.display(board)
        time.sleep(0.5)

A helper method `log_move` is implemented to write one move to a file. It takes the current chess board `board` as a parameter, the move `move` that should be logged, the name of the file `file_name` to log into and a boolean flag `log`. Similarly to before, the method will not do anything if `log` is set to `False`. Otherwise, it writes the move into the file.

In [ ]:
def log_move(
    board: chess.Board, move: chess.Move, file_name: str, log: bool = True
):
    if log:
        with open(file_name, "a") as f:
            if board.turn is chess.WHITE:
                f.write(board.lan(move) + "\t")
            else:
                f.write(board.lan(move) + "\n")

Next, we can define the actual `play_game` method to let two engines play each other. As a parameter, it takes the start position `board`, two engines `engine1` and `engine2` as well as two boolean flags `display_board` and `log_moves` to decide whether the board should be displayed after each half turn and the moves should be logged to a file, respectively.

In [ ]:
from converted_notebooks.s04_engine_interface import Engine
from datetime import datetime


def play_game(
    board: chess.Board,
    engine1: Engine,
    engine2: Engine,
    display_board: bool = False,
    log_moves: bool = False
) -> chess.Board():
    engines = [engine1, engine2]

    log_file_ame = f"../../log/{datetime.now().strftime('%Y%m%d-%H%M%S')}.txt"

    display_chess_board(board, display_board)

    while not board.is_game_over():
        move = engines[0].play(board).move

        log_move(board, move, log_file_ame, log_moves)

        board.push(move)
        engines[0], engines[1] = engines[1], engines[0]

        display_chess_board(board, display_board)
    return board

We can test the function by letting two random engines play against each other. By passing `display_board = True` one can see very clearly that both play randomly and without any strategy.

In [ ]:
import random
from converted_notebooks.s05_random_engine import RandomEngine

random.seed(42)

board = chess.Board()
play_game(board, RandomEngine(), RandomEngine(), display_board=True)
print(board.outcome())

As described earlier, the `Engine` interface is similar to the one defined by the `python-chess` library, but with fewer details that are not needed for our engines. To still be able to play against external engines with an uci interface, a wrapper class `UciEngine` that inherits `Engine` is introduced.

In [ ]:
class UciEngine(Engine):

    def __init__(
        self,
        engine_executable: str = "stockfish",
        limit: chess.engine.Limit = chess.engine.Limit(time=0.1)
    ):
        self.limit = limit
        self.engine = chess.engine.SimpleEngine.popen_uci(engine_executable)

    def play(self, board: chess.Board) -> chess.engine.PlayResult:
        return self.engine.play(board, self.limit)

We can then let the random engine play against `stockfish`.

In [ ]:
import random
from converted_notebooks.s05_random_engine import RandomEngine

random.seed(42)

board = chess.Board()
play_game(board, RandomEngine(), UciEngine(), display_board=True)
print(board.outcome())